In [9]:
%pip install -q motor

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import asyncio
from motor.motor_asyncio import AsyncIOMotorClient

In [3]:
from db.db_main import DarkstarDb, collection_weapons,collection_ability_mechanics, collection_consumables,collection_skyship_upgrades,collection_abilities, collection_devs_data, collection_players_data

Successfully connected to MongoDB!


In [ ]:
async def update_collection_fields(collection, fields, target_type=int):
    """
    Met à jour les champs spécifiés dans tous les documents d'une collection en les convertissant vers un type donné.
    
    :param collection: La collection MongoDB à mettre à jour.
    :param fields: Liste des champs à convertir (ou un seul champ sous forme de string).
    :param target_type: Type de destination (int, float, str, etc.).
    """
    # Vérifier et transformer fields en liste si nécessaire
    if isinstance(fields, str):
        fields = [fields]  # Convertir un seul champ string en liste
    elif not isinstance(fields, list):
        print(f"❌ ERREUR: 'fields' doit être une liste ou une chaîne de caractères. Type reçu: {type(fields)}")
        return

    print(f"🚀 Début de la mise à jour des champs {fields} vers {target_type.__name__}...\n")

    try:
        # Boucle asynchrone pour parcourir les documents
        async for doc in collection.find():
            try:
                doc_id = doc["_id"]  # ID du document
                update_fields = {}

                print(f"🔍 Traitement du document ID: {doc_id}")

                for field in fields:
                    if field in doc:
                        value = doc[field]

                        # Vérification de la valeur avant de la convertir
                        if value is None or (isinstance(value, str) and value.strip() == ''):
                            print(f"⚠️ Le champ '{field}' est vide ou invalide dans le document ID: {doc_id}. Ignorer cette valeur.")
                            continue

                        try:
                            # Tentative de conversion de la valeur
                            converted_value = target_type(value)
                            update_fields[field] = converted_value
                            print(f"✅ Conversion du champ '{field}' avec valeur '{value}' en {target_type.__name__} pour le document ID: {doc_id}")
                        except ValueError:
                            print(f"⚠️ Impossible de convertir '{value}' en {target_type.__name__} pour {field} (ID: {doc_id})")

                # Appliquer la mise à jour si des champs ont été modifiés
                if update_fields:
                    print(f"🔨 Mise à jour du document ID: {doc_id} avec {update_fields}")
                    await collection.update_one({"_id": doc_id}, {"$set": update_fields})
                    print(f"✅ Document ID: {doc_id} mis à jour avec {update_fields}\n")
                else:
                    print(f"ℹ️ Aucune mise à jour nécessaire pour le document ID: {doc_id}\n")

            except Exception as e:
                print(f"❌ Erreur lors du traitement du document ID: {doc_id} -> {e}")

    except Exception as e:
        print(f"❌ Erreur générale dans update_collection_fields(): {e}")

    print("\n🎯 Mise à jour terminée !")

# Exécuter dans Jupyter pour la collection "WEAPONS"
# Appeler la fonction pour la collection "WEAPONS" et convertir le champ "Cooldown (in turns)" en entier
await update_collection_fields(collection_weapons, fields=["Cooldown (in turns)"], target_type=int)


🚀 Début de la mise à jour des champs ['Cooldown (in turns)'] vers int...


🎯 Mise à jour terminée !
